# 欢迎语
更新日志

## <i class="fa fa-repeat"></i> 切换pip软件源

In [ ]:
import os
import subprocess

def check_pip_list():
    output = !pip config list
    if "global.index-url='https://pypi.tuna.tsinghua.edu.cn/simple'" in output and "global.trusted-host='pypi.tuna.tsinghua.edu.cn'" in output:
        return True
    else:
        return False
if not check_pip_list():
    # 查找pip.conf文件
    pip_conf_file = os.popen("find / -name 'pip.conf'").read().strip()
    # 直接覆盖pip.conf文件
    with open(pip_conf_file, 'w') as f:
        f.write("[global]\nindex-url = https://pypi.tuna.tsinghua.edu.cn/simple\ntrusted-host = pypi.tuna.tsinghua.edu.cn")
    # 验证修改是否成功
    if check_pip_list():
        print("pip软件源切换成功！")
    else:
        print("pip软件源切换失败，请检查脚本是否正确运行。")
else:
    print("pip软件源本来就是清华源，无需切换")

##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 判断当前脚本目录是否存在`.git`文件夹
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
print(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)
# 获取当前环境名称与当前环境content路径
from func.env import detect_environment
env_cb = detect_environment()
env_name = env_cb['env_name']
content_path = env_cb['content_path']

## <i class="fa fa-arrow-down"></i> 安装必要依赖

In [ ]:
import subprocess

libraries_to_install = [
    
]

tools_to_install = [
    "iputils-ping",
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install $library
    else:
        print(f"【 {library} 】已经安装，跳过安装")
        
# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool
    


# 基础配置

In [ ]:
#需要安装ipywidgets，此镜像已经安装了对应的依赖包
# !conda install -c conda-forge jupyterlab 
# !conda install -c conda-forge ipywidgets
# !conda install -c conda-forge ipython
# !conda install -c conda-forge ipympl

# 实现显示/隐藏代码块的步骤：
# 安装jupyter_contrib_nbextentions
# （Ref: https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html#install-the-python-package）
# conda安装命令：conda install -c conda-forge jupyter_contrib_nbextensions
# python安装命令：pip install jupyter_contrib_nbextensions

# 安装jupyter_nbextentions_contributor，以便在Jupyter Notebook页面设置nbextentions
# （Ref: https://github.com/Jupyter-contrib/jupyter_nbextensions_configurator）
# conda安装命令：conda install -c conda-forge jupyter_nbextensions_configurator
# python安装命令：pip install jupyter_nbextensions_configurator

# !apt-get update && apt install net-tools

## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
import sys
sys.path.append('../') # 因为func与ipynb位于同一个目录下，所以要往上一层路径索引
from func.env import setProxy
cb=setProxy()
proxy=cb['proxy']
region=cb['region']

## <i class="fa fa-star"></i> Notebook环境切换
在JupyterLab的Notebook中使用新的Conda环境

In [ ]:
from subprocess import getoutput
from IPython.display import clear_output
import ipywidgets as widgets

s = getoutput('conda info --env')
print(s)
if 'py3.10.6' not in s:
    print('当前系统中没有py3.10.6环境，正在安装')
    # 创建Conda新的虚拟环境
    # 构建一个虚拟环境，名为: py3.10.6
    # 更新bashrc中的环境变量
    !source /usr/local/etc/profile.d/conda.sh &&\
    conda create -n py3.10.6 python=3.10.6 -y &&\
    conda init bash && source /root/.bashrc -y
else:
    print('已经安装了py3.10.6环境')
    # !source activate &&\
    # conda activate py3.10.6 &&\
    # conda install ipykernel &&\
    # ipython kernel install --user --name=py3.10.6 --display-name "Python3.10.6" # 设置kernel，--user表示当前用户，--name为虚拟环境名称，--display-name为笔记本显示名称

# clear_output(wait=True)
# print("Done!")

widgets.Button(
    value=False,
    description='环境部署成功!',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

## <i class="fa fa-download"></i> 更新本体与环境

更新Stable-diffusion-WebUI本体与环境命令

In [ ]:
from IPython.display import display,clear_output
clear_output()
import ipywidgets as widgets
from func.env import setProxy,getWebUIDir

try:
    proxy,region
except NameError:
    cb=setProxy()
    proxy=cb['proxy']
    region=cb['region']
    clear_output(wait=True)

webUIDir=getWebUIDir()
print(webUIDir)

base_args = [
    'source activate','&&',
    'conda activate py3.10.6','&&',
    proxy,'&&'
       ]

debugging=False

alertBtn = widgets.Button(
    value=False,
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='' # (FontAwesome names without the `fa-` prefix)
)


display(alertBtn)
    
if os.path.exists(webUIDir):
    
    alertBtn.description='正在更新...'
    alertBtn.button_style='warning'
    alertBtn.icon='upload'
    
    update_sd_webui_args = base_args[:]
    update_sd_webui_args += [
        f'cd {webUIDir}','&&',
        'git pull','&&',
        'python -V','&&',
        'python -m pip install -r requirements.txt'
    ]
    update_sd_webui_cmd = (" ").join(update_sd_webui_args)
    !$update_sd_webui_cmd

    if debugging==False:
        clear_output()
        display(alertBtn)
    
    alertBtn.description='更新完成!'
    alertBtn.button_style='success'
    alertBtn.icon='check'

else:
    alertBtn.description='正在安装SDWebUI...'
    alertBtn.button_style='info'
    alertBtn.icon='download'
    sd_webui_download_args = base_args[:]
    sd_webui_download_args += [
        f'cd {root_dir}','&&',
        'git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui','&&',
        'python -V','&&',
        'python -m pip install -r requirements.txt'
    ]
    sd_webui_download_cmd = (" ").join(sd_webui_download_args)
    !$sd_webui_download_cmd
    
#     r = !source activate &&\
#     conda activate py3.10.6 &&\
#     $proxy &&\
#     cd $webUIDir &&\
#     git pull &&\
#     python -V &&\
#     python -m pip install -r requirements.txt

#     display(r)
    
#     if 'Already up to date.' in r:
#         alertBtn.description='更新完成!'
#         alertBtn.button_style='success'
#         alertBtn.icon='check'
#     else:
#         alertBtn.description='更新失败!'
#         alertBtn.button_style='danger'
#         alertBtn.icon='exclamation'


# clear_output(wait=True)

## <i class="fa fa-bolt"></i> 安装Xformers

大幅减少计算时间和显存占用，代价是生成的图每个都有点变化，强烈推荐安装!  
但不同核心代号的显卡需要不同的编译包，不能混用  
当然此笔记会帮你自动检测这台机器是否适合安装，而且会自动帮你安装合适的编译包

In [ ]:
xformers_install_args = base_args[:]

xformers_install_args += [
    f'cd {webUIDir}','&&',
    'conda install xformers -c xformers/label/dev -y'
]

xformers_install_cmd = (" ").join(xformers_install_args)

!$xformers_install_cmd

In [ ]:
# from func.xformers import installXformers
# from func.env import setProxy
# from IPython.display import clear_output


# try:
#     proxy,region
# except NameError:
#     cb=setProxy()
#     proxy=cb['proxy']
#     region=cb['region']
#     clear_output(wait=True)

# installXformers(proxy,region)

## <i class="fa fa-language"></i> 安装中文语言包

In [ ]:
from func.extensionsInstaller import install

extURL='https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN'
extFileSize=171055

install(extURL,extFileSize)

## <i class="fa fa-images"></i> 安装图库浏览器

In [ ]:
from func.extensionsInstaller import install

extURL='https://github.com/yfszzx/stable-diffusion-webui-images-browser'
extFileSize=82945

install(extURL,extFileSize)

## 挂载.ckpt到SD WebUI模型文件夹

In [ ]:
import os
from func.checksum import generate_checksum,find_folder

# def find_file(root_dir, target_name):
#     paths = []
#     paths = !find $root_dir -name $target_name
#     print(paths)
    
#     paths.sort(key=len)
#     if paths:
#         return paths
#     else:
#         return None
def create_symlink(source_path, symlink_path, symlink_name):
    # 将软链接路径拼接成完整的文件路径
    symlink_file = f"{symlink_path}/{symlink_name}"

    # 如果软链接文件存在，则先删除它
    if os.path.exists(symlink_file):
        os.remove(symlink_file)

    # 创建软链接
    os.symlink(source_path, symlink_file)

    # 验证软链接是否成功创建
    if os.path.islink(symlink_file):
        print(f"{symlink_file} 是一个符号链接。")
        print(f"\033[1m\033[92m{symlink_file}\033[0m ➜ \033[34m{os.readlink(symlink_file)}\033[0m")
    else:
        print(f"{symlink_file} 不是一个符号链接。")

def find_file(root_dir, target_name):
    paths = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == target_name:
                path = os.path.join(dirpath, filename)
                paths.append(path)
    paths.sort(key=len)
    if paths:
        return paths
    else:
        return None

def check_model_and_make_symbollink():
    
    stable_diffusion_v2_dir = find_file('/', 'v2-1_768-nonema-pruned.ckpt')[0]

    if stable_diffusion_v2_dir:
        print(stable_diffusion_v2_dir)
        webUIDir = getWebUIDir()
        sd_models_dir = os.path.realpath(os.path.join(webUIDir,'./models/Stable-diffusion'))
        model_path = os.path.realpath(os.path.join(sd_models_dir,'v2-1_768-nonema-pruned.ckpt'))

        if stable_diffusion_v2_dir==model_path:
            print('stable-diffusion-v2的diffuser形态的模型已经在目标位置，无需创建软连接')
        else:
            # 在Content目录下创建一个指向挂载模型的软连接
            create_symlink(
                source_path=stable_diffusion_v2_dir, # openbayes 特有的目录，是外部数据集挂载到当前镜像的目录，编号从0~4
                symlink_path='/output/stable-diffusion-webui/models/Stable-diffusion', # fast-dreambooth.ipynb的 model download cell一般下载模型的路径
                symlink_name='v2-1_768-nonema-pruned.ckpt' # 模型名称
            )
    # else:
    #     print('找不到stable-diffusion-v2的diffuser形态的模型，尝试通过运行convertodiffv2.ipynb来获取模型')
    #     convertodiffv2_path = os.path.join(repo_root_dir,'./tools/model_convert_tools/convertodiffv2.ipynb')
    #     %run $convertodiffv2_path
    #     check_model_and_make_symbollink()

check_model_and_make_symbollink()

# <i class="fa fa-rocket"></i> 启动Stable-Diffusion-WebUI

In [ ]:
from func.env import setProxy,getWebUIDir,getExtDir,findDir,findFile,getDirSize,styleURL,getArch
import ipywidgets as widgets
from subprocess import getoutput
from func.xformers import checkXformersInstalled
from IPython.display import display,clear_output

extDir=getExtDir()
archCondition = False
canUseXformers = False

# cb = !source activate &&\
# conda activate py3.10.6 &&\
# python getArch.py

cb = getArch()
arch = cb['arch']
gpu = cb['gpu']

whiteList = ['sm80','sm75','sm70','sm86','sm89','sm61']

if arch in whiteList:
    archCondition = True
else:
    archCondition = False
        
if(checkXformersInstalled() and archCondition):
    canUseXformers = True
else:
    canUseXformers = False

canUseXformers = True

webUIDir = getWebUIDir()

args = [
        'source activate','&&',
        'conda activate py3.10.6','&&',
        f'cd {webUIDir}','&&',
        'python launch.py',
        '--share',
        '--opt-split-attention',
        '--port 7860',
        '--deepdanbooru',
        '--administrator',
        '--disable-safe-unpickle',
        '--no-half'
       ]

try:
    proxy
except NameError:
    print("不存在proxy")
else:
    if region=='未知':
        print("不存在proxy")
    else:
        print(f"已经挂载【{region}】proxy")
        args.insert(0,f"{proxy} && env|grep -i proxy &&")

# if(canUseXformers==True):
#     print("可以使用Xformers")
#     args.append("--xformers")
# else:
#     print(f"暂不支持【{gpu}】该型号GPU，不使用Xformers")
    
cmd = (" ").join(args)

!$cmd